# 基于昇思MindSpore+Orangepi AIpro的CNNCTC文字识别

文本识别指从图像中识别出文本，将图像中的文字区域转化为字符信息，通常采用CNN网络从图像中提取丰富的特征信息，然后根据提取的特征信息进行识别。这里采用ResNet作为特征提取网络，采用CTC(Connectionist Temporal Classification)方法进行识别。
此脚本将**基于昇思MindSpore框架开发的cnnctc模型训练出来的权重文件转换成OM文件**后，调用ACL相关接口进行离线推理。

## 前期准备

* 基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可以参考下面的步骤。
* **建议在Linux服务器或者虚拟机转换该模型。**
* **为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用；转换指导详见全流程实验指导。**

In [ ]:
## 环境配置

In [1]:
!sh env.sh

## 模型推理实现 ##
得到cnnctc.om后，执行离线推理代码,加载图片predict.png

![jupyter](./predict.png)

### 1. 导入三方库

In [1]:
import os

import time
import argparse

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from acllite_model import AclLiteModel as Model
from acllite_resource import AclLiteResource as AclResource

### 2. 模型导入和处理

In [ ]:
# 获取模型om文件
from download import download
model_url = "https://modelers.cn/coderepo/web/v1/file/MindSpore-Lab/cluoud_obs/main/media/examples/mindspore-courses/orange-pi-mindspore/02-CNNCTC/cnnctc.zip"
download(model_url, "./", kind="zip", replace=True)

In [2]:
# om模型和图片的位置
MODEL_PATH = './cnnctc.om'
IMAGE_PATH = './predict.png'

# 初始化acl资源
acl_resource = AclResource()
acl_resource.init()

#导入本地om模型
print('load model....')
model = Model(MODEL_PATH)
print('load model finished....')

# 文本与数据编码
class CTCLabelConverter():
    def __init__(self, character):
        dict_character = list(character)
        self.dict = {}
        for i, char in enumerate(dict_character):
            self.dict[char] = i + 1
        self.character = ['[blank]'] + dict_character
        self.dict['[blank]'] = 0

    #将文本转换为数字编码
    def encode(self, text):
        length = [len(s) for s in text]
        text = ''.join(text)
        text = [self.dict[char] for char in text]

        return np.array(text), np.array(length)

    # 将数字编码转换为文本
    def decode(self, text_index, length):
        texts = []
        index = 0
        for l in length:
            t = text_index[index:index + l]
            char_list = []
            for i in range(l):
                if t[i] != self.dict['[blank]'] and (
                        not (i > 0 and t[i - 1] == t[i])):
                    char_list.append(self.character[t[i]])
            text = ''.join(char_list)
            texts.append(text)
            index += l
        return texts


init resource stage:
Init resource success
load model....
Init model resource start...
[AclLiteModel] create model output dataset:
malloc output 0, size 3848
Create model output dataset success
Init model resource success
load model finished....


### 3. 进行推理

In [3]:
# 导入和处理目标图片
img_PIL = Image.open(IMAGE_PATH).convert('RGB')
img = img_PIL.resize((100, 32), resample=3)
img = np.array(img, dtype=np.float32)
img = np.expand_dims(img, axis=0) 
img = np.transpose(img, [0, 3, 1, 2]) 

# 定义推理的时间
start = time.time()
model_predict = model.execute([img])[0]
end = time.time()
print(f'infer use time:{(end-start)*1000}ms')

# 初始化文本编码函数
character = '0123456789abcdefghijklmnopqrstuvwxyz'
converter = CTCLabelConverter(character)

# 推理过程
preds_size = np.array([model_predict.shape[1]])
preds_index = np.argmax(model_predict, 2)
preds_index = np.reshape(preds_index, [-1])
preds_str = converter.decode(preds_index, preds_size)
print('Predict: ', preds_str)

infer use time:8.622884750366211ms
Predict:  ['parking']


## 总结与扩展 ##

以上就是cnnctc文本识别样例离线推理的运行结果了，可以看到最后的验证结果，成功识别了示例图片中‘PARKING’的字样。
注意：
1. 若出现推理失败的情况，请确保以root权限设置好环境变量（运行或参考文件夹内的env.sh文件）。
2. 再次进行推理清清空所有缓存。